In [1]:
from dotenv import load_dotenv
import requests
import os
import json
from collections import Counter

load_dotenv(".env")

os.makedirs("output", exist_ok=True)

In [39]:
with open("queries/search_queries_ru.json", "r") as f:
    queries_ru = json.load(f)
with open("queries/search_queries.json", "r") as f:
    queries_en = json.load(f)

queries_list_ru = [{"query": q, "category": cat} for cat in queries_ru for q in queries_ru[cat]]
queries_list_en = [{"query": q, "category": cat} for cat in queries_en for q in queries_en[cat]]

In [40]:
# get date of 7 days before today
from datetime import datetime, timedelta
date = datetime.today() - timedelta(days=7)
date = date.strftime("%Y-%m-%d")

In [36]:
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='80f585e493484c3abf0a8b946ddccf5d')

In [52]:
url_items = []

In [ ]:
lang = "ru"

for i, query_d in enumerate(queries_list_ru):
    query = query_d["query"]
    category = query_d["category"]

    res = newsapi.get_everything(
        q=query,
        from_param=date,
        language=lang,
        sort_by='relevancy',
        # page_size=100,
        page=1
    )
    print(f"Searching for {query}, {i}/{len(queries_list_ru)}, found {res['totalResults']} articles")
    articles = res["articles"]

    for item in articles:
        item.pop("content")
        url_items.append({
            **item,
            "query": query,
            "category": category,
        })

In [53]:
lang = "en"

for i, query_d in enumerate(queries_list_en):
    query = query_d["query"]
    category = query_d["category"]

    res = newsapi.get_everything(
        q=query,
        from_param=date,
        language=lang,
        sort_by='relevancy',
        # page_size=100,
        page=1
    )
    print(f"Searching for {query}, {i}/{len(queries_list_en)}, found {res['totalResults']} articles")
    articles = res["articles"]

    for item in articles:
        item.pop("content")
        url_items.append({
            **item,
            "query": query,
            "category": category,
        })

Searching for Latest products in the steel industry, 0/69, found 69 articles
Searching for New initiatives by leading steel companies, 1/69, found 12 articles
Searching for Competitor analysis steel industry, 2/69, found 1 articles
Searching for Innovations in steel production, 3/69, found 11 articles
Searching for New steel products market trends, 4/69, found 34 articles
Searching for Innovative data storage solutions for the steel industry, 5/69, found 3 articles
Searching for Data management trends in metallurgy, 6/69, found 4 articles
Searching for Best practices for data storage in manufacturing, 7/69, found 13 articles
Searching for Cloud solutions for steel manufacturing, 8/69, found 5 articles
Searching for Big data in steel industry, 9/69, found 15 articles
Searching for AI applications in steel industry, 10/69, found 8 articles
Searching for Artificial intelligence in metallurgy, 11/69, found 1 articles
Searching for Top startups in steel and metallurgy, 12/69, found 0 articl

NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [54]:
# Save URLs
with open("output/urls_newsapi.json", "w") as f:
    json.dump(url_items, f, indent=2, ensure_ascii=False)

## Filter and Count Duplicates

In [59]:
def deduplicate_urls(url_items):
    urls_dedupl = {}

    urls = [item['url'] for item in url_items]
    counter = Counter(urls)
    for item in url_items:
        url = item['url']
        if urls_dedupl.get(url):
            continue
        urls_dedupl[url] = item
        urls_dedupl[url]['count'] = counter[url]

    assert len(urls_dedupl) == len(counter)
    return list(urls_dedupl.values())

In [62]:
url_items = deduplicate_urls(url_items)

with open("output/urls_newsapi_filtered.json", "w") as f:
    json.dump(url_items, f, indent=2, ensure_ascii=False)

In [63]:
len(url_items)

205